# **Image Matching Pipeline using SIFT + LightGlue**

Here's a detailed explanation of your **image matching pipeline** using SIFT + LightGlue in PyTorch/Kornia:

---

### **1️⃣ Imports and Setup**

```python
import os, io, requests, cv2, torch, kornia as K, kornia.feature as KF
from kornia_moons.feature import laf_from_opencv_SIFT_kpts
from kornia_moons.viz import draw_LAF_matches
import matplotlib.pyplot as plt
```

* **OpenCV (cv2)**: for SIFT keypoint detection.
* **Kornia / Kornia-Moons**: differentiable computer vision in PyTorch. Includes LightGlue matcher, LAF utilities, and visualization helpers.
* **Torch**: core tensor operations, device handling.
* **Matplotlib**: visualize matches.

---

### **2️⃣ Device Selection**

```python
compute_device = K.utils.get_cuda_or_mps_device_if_available()
print(compute_device)
```

* Chooses GPU if available (CUDA or Apple MPS).
* Speeds up descriptor matching and LightGlue computations.

---

### **3️⃣ LightGlue Matcher and SIFT Initialization**

```python
lightglue_model = KF.LightGlueMatcher("sift").eval().to(compute_device)
max_keypoints = 4096
sift_detector = cv2.SIFT_create(max_keypoints)
```

* **LightGlueMatcher("sift")**: pretrained model for SIFT descriptors.
* **SIFT_create(4096)**: detects up to 4096 features per image.

---

### **4️⃣ Image Loading**

```python
image_a = cv2.cvtColor(cv2.imread(img_path_1), cv2.COLOR_BGR2RGB)
image_b = cv2.cvtColor(cv2.imread(img_path_2), cv2.COLOR_BGR2RGB)
dims_a = torch.tensor(image_a.shape[:2], device=compute_device)
dims_b = torch.tensor(image_b.shape[:2], device=compute_device)
```

* Reads and converts OpenCV BGR → RGB.
* Stores height/width for LightGlue scaling.

---

### **5️⃣ Descriptor Normalization: RootSIFT**

```python
def convert_to_rootsift(desc: torch.Tensor, epsilon=1e-6) -> torch.Tensor:
    desc = torch.nn.functional.normalize(desc, p=1, dim=-1, eps=epsilon)
    desc.clip_(min=epsilon).sqrt_()
    return torch.nn.functional.normalize(desc, p=2, dim=-1, eps=epsilon)
```

* **RootSIFT** improves SIFT matching by:

  1. L1-normalizing the descriptor.
  2. Taking square root (Hellinger kernel).
  3. L2-normalizing again.

---

### **6️⃣ Detect Keypoints and Compute Descriptors**

```python
with torch.inference_mode():
    keypoints_a, descriptors_a = sift_detector.detectAndCompute(image_a, None)
    keypoints_b, descriptors_b = sift_detector.detectAndCompute(image_b, None)
```

* Uses OpenCV SIFT to extract **keypoints** and **descriptors**.

```python
frames_a = laf_from_opencv_SIFT_kpts(keypoints_a, compute_device)
frames_b = laf_from_opencv_SIFT_kpts(keypoints_b, compute_device)
descriptors_a = convert_to_rootsift(torch.from_numpy(descriptors_a)).to(compute_device)
descriptors_b = convert_to_rootsift(torch.from_numpy(descriptors_b)).to(compute_device)
```

* Converts OpenCV keypoints → **Local Affine Frames (LAFs)** compatible with Kornia.
* Converts descriptors to PyTorch and applies **RootSIFT**.

---

### **7️⃣ LightGlue Matching**

```python
distances, match_indices = lightglue_model(descriptors_a, descriptors_b, frames_a, frames_b, hw1=dims_a, hw2=dims_b)
```

* LightGlue produces **tentative matches** between two sets of descriptors using attention-based matching.
* `match_indices` contains matched indices: `(idx_image_a, idx_image_b)`.

```python
print(f"{match_indices.shape[0]} tentative matches with SIFT-LightGlue")
```

---

### **8️⃣ Extract Matched Keypoints**

```python
def extract_matched_points(points_a, points_b, match_indices):
    matched_pts_a = points_a[match_indices[:, 0]]
    matched_pts_b = points_b[match_indices[:, 1]]
    return matched_pts_a, matched_pts_b

matched_pts_a, matched_pts_b = extract_matched_points(KF.get_laf_center(frames_a)[0], KF.get_laf_center(frames_b)[0], match_indices.detach().cpu())
```

* Converts LAFs → **keypoint centers**.
* Retrieves matched coordinates for both images.

---

### **9️⃣ Estimate Fundamental Matrix + Inliers**

```python
fundamental_mat, inlier_mask = cv2.findFundamentalMat(
    matched_pts_a.detach().cpu().numpy(), matched_pts_b.detach().cpu().numpy(),
    cv2.USAC_MAGSAC, 1.5, 0.999, 100000
)
inlier_mask = inlier_mask > 0
```

* **RANSAC-based fundamental matrix estimation**.
* Filters out outliers to keep only **geometrically consistent matches**.

---

### **🔟 Visualize Matches**

```python
draw_LAF_matches(
    frames_a, frames_b,
    match_indices.cpu().numpy(),
    image_a, image_b,
    inlier_mask,
    draw_dict={
        "inlier_color": (0.2, 1, 0.2),
        "tentative_color": (1, 1, 0.2, 0.3),
        "feature_color": None,
        "vertical": False
    },
)
plt.title(f"{inlier_mask.sum()} inliers with SIFT-LightGlue")
```

* Draws tentative matches (yellow) and **inliers** (green) between images.
* LAFs allow visualization of **keypoint affine frames**, not just points.

---

### ✅ **Pipeline Summary**

1. Load images and initialize SIFT.
2. Detect keypoints & compute SIFT descriptors.
3. Convert keypoints → LAFs (affine frames).
4. Convert descriptors → RootSIFT + PyTorch tensors.
5. Run **LightGlue** to get tentative matches.
6. Extract keypoint coordinates from LAFs.
7. Compute **fundamental matrix** to filter inliers.
8. Visualize matches with LAFs and inlier overlay.

This approach combines **classic SIFT features** with **modern LightGlue attention-based matching**, giving robust matches even under scale, rotation, and lighting changes.

---

If you want, I can also **draw a step-by-step diagram** showing this pipeline visually from images → SIFT → LightGlue → inliers. It helps a lot for presentations. Do you want me to do that?

In [ ]:
# Using LightGlue via kornia requires kornia>=0.7.2
!pip install kornia==0.7.2
!pip install kornia_moons --no-deps

In [ ]:
!mkdir -p .cache/torch/hub/checkpoints
!cp /kaggle/input/lightglue/pytorch/sift/1/sift_lightglue.pth .cache/torch/hub/checkpoints/

In [ ]:
import os
import io
import requests
import cv2
import kornia as K
import kornia.feature as KF
import matplotlib.pyplot as plt
import torch
from kornia_moons.feature import laf_from_opencv_SIFT_kpts
from kornia_moons.viz import draw_LAF_matches
import numpy as np

In [ ]:
img_path_1 = "/kaggle/input/image-matching-challenge-2024/train/church/images/00017.png"
img_path_2 = "/kaggle/input/image-matching-challenge-2024/train/church/images/00020.png"

In [ ]:
def convert_to_rootsift(desc: torch.Tensor, epsilon=1e-6) -> torch.Tensor:
    desc = torch.nn.functional.normalize(desc, p=1, dim=-1, eps=epsilon)
    desc.clip_(min=epsilon).sqrt_()
    return torch.nn.functional.normalize(desc, p=2, dim=-1, eps=epsilon)

compute_device = K.utils.get_cuda_or_mps_device_if_available()
print(compute_device)


lightglue_model = KF.LightGlueMatcher("sift").eval().to(compute_device)

max_keypoints = 4096
sift_detector = cv2.SIFT_create(max_keypoints)

image_a = cv2.cvtColor(cv2.imread(img_path_1), cv2.COLOR_BGR2RGB)
image_b = cv2.cvtColor(cv2.imread(img_path_2), cv2.COLOR_BGR2RGB)

dims_a = torch.tensor(image_a.shape[:2], device=compute_device)
dims_b = torch.tensor(image_b.shape[:2], device=compute_device)


with torch.inference_mode():
    keypoints_a, descriptors_a = sift_detector.detectAndCompute(image_a, None)
    keypoints_b, descriptors_b = sift_detector.detectAndCompute(image_b, None)
    frames_a = laf_from_opencv_SIFT_kpts(keypoints_a, compute_device)
    frames_b = laf_from_opencv_SIFT_kpts(keypoints_b, compute_device)
    descriptors_a = convert_to_rootsift(torch.from_numpy(descriptors_a)).to(compute_device)
    descriptors_b = convert_to_rootsift(torch.from_numpy(descriptors_b)).to(compute_device)
    distances, match_indices = lightglue_model(descriptors_a, descriptors_b, frames_a, frames_b, hw1=dims_a, hw2=dims_b)

print(f"{match_indices.shape[0]} tentative matches with SIFT-LightGlue")

In [ ]:
from PIL import Image
preview_a = Image.open(img_path_1)
preview_b = Image.open(img_path_2)
display(preview_a)
display(preview_b)

In [ ]:
def extract_matched_points(points_a, points_b, match_indices):
    matched_pts_a = points_a[match_indices[:, 0]]
    matched_pts_b = points_b[match_indices[:, 1]]
    return matched_pts_a, matched_pts_b


matched_pts_a, matched_pts_b = extract_matched_points(KF.get_laf_center(frames_a)[0], KF.get_laf_center(frames_b)[0], match_indices.detach().cpu())

fundamental_mat, inlier_mask = cv2.findFundamentalMat(
    matched_pts_a.detach().cpu().numpy(), matched_pts_b.detach().cpu().numpy(), cv2.USAC_MAGSAC, 1.5, 0.999, 100000
)
inlier_mask = inlier_mask > 0

draw_LAF_matches(
    frames_a, 
    frames_b,
    match_indices.cpu().numpy(),
    image_a,
    image_b,
    inlier_mask,
    draw_dict={"inlier_color": (0.2, 1, 0.2), 
               "tentative_color": (1, 1, 0.2, 0.3), 
               "feature_color": None, 
               "vertical": True},
)
plt.title(f"{inlier_mask.sum()} inliers with SIFT-LightGlue")

    Inliers
    Correct correspondences (shown in green)
    Reliable correspondences that are geometrically consistent

    Outliers
    Incorrect correspondences (remain yellow)
    Incorrect correspondences that are geometrically inconsistent